In [1]:
import pandas as pd
import sys
import matplotlib.pyplot as plt
import numpy as np
import math
import datetime

# Global Dictionarys
parkInfo = {}
Inspections = {}

PIP_InspectionMain = pd.ExcelFile("InspectionMain.xlsx")
inspection = PIP_InspectionMain.parse()

PIP_FeatureRating = pd.ExcelFile("FeatureRatings.xlsx")
rating = PIP_FeatureRating.parse()

PIP_AllSites = pd.ExcelFile("ALLSITES.xlsx")
sites = PIP_AllSites.parse()

failedFeatures = rating[rating['Rating'].isin(['U'])]
badFeatures = rating[rating['Rating'].isin(['U/S'])]

'''
Example Structures:
parkID = {<PARKID>: 
            {'Acres':0, 'Name':0, 'Borough':0, 'District':0, 'Category':0, 'Inspections':
                                                                                {<INSPECTIONID> : {'Count':0, 'Failures':0, 'Date':date, 'Rating':0}},
                                                                                {<INSPECTIONID> : {'Count':0, 'Failures':0, 'Date':date, 'Rating':0}},
                                                                                {<INSPECTIONID> : {'Count':0, 'Failures':0, 'Date':date, 'Rating':0}}
            }
          }
'''


#  Iterate through all inspection reports and sum Ratio of Failures of features to Features themselves.
#  Maintain 'Inspection' dictionary of format Inspection[<INSPECTIONID>]: {'Count':X, 'Failures':X}
for feature in rating.iterrows():
    inspectionID = feature[1][2]
    featureRating =feature[1][1]
    
    if Inspections.get(inspectionID) == None:
        Inspections[inspectionID] = {'Count':0, 'Failures':0}
    
    if featureRating in ['U','U/S']:
        Inspections[inspectionID]['Failures']+=1
        
    Inspections[inspectionID]['Count']+=1

'''
# Try this instead of above loop.  Might be more efficient.  Mostly just more 'python' like. :)
for inspectionID in pd.Series(sites[2]).unique():
  # Slice Inspection report
  InspectionDf = sites[sites[2] == inspectionID]
  # Create Masks
  RatingFilterU = InspectionDf[1] == 'U'
  RatingFilterUS = InspectionDf[1] == 'U/S'
  # Build Dictionary Entry
  Inspections[inspectionID] = {}
  Inspections[inspectionID]['Count'] = InspectionDf.count()
  Inspections[inspectionID]['Failures'] = InspectionDf[RatingFilterU | RatingFilterUS]
  Inspections[inspectionID]['Ratio'] = float(Inspections[inspectionID]['Failures']) / Inspections[inspectionID]['Count']
'''

for feature in sites.iterrows():
    parkID = feature[1][1]
    borough = feature[1][2]
    district = feature[1][4]
    name = feature[1][6]
    acres = feature[1][9]
    category = feature[1][10]
    
    if parkInfo.get(parkID) == None:
        parkInfo[parkID] = {'Inspections':{}}

    parkInfo[parkID]['Acres']=float(acres)
    parkInfo[parkID]['Name']=name
    parkInfo[parkID]['Borough']=borough
    parkInfo[parkID]['District']=district
    parkInfo[parkID]['Category']=category
    
for feature in inspection.iterrows():
    parkID = feature[1][0]
    date = feature[1][3]
    inspectionID = feature[1][11]

    '''SEE if we can just read off the dataframe instead of going through the above for loop
    # Read entry 0... just in case there is more than one entry per parkID.. There shouldn't be regardless
    parkDfEntry = sites[sites[1] == parkID].ix[0]
    if parkDfEntry.empty:
      print 'No parkID matches %s found in Inspection file. SKIPPING inspectionID %s!' % (parkID, inspectionID)
    else:
      if parkInfo.get(parkID) == None:
          parkInfo[parkID] = {'Inpections':{}}
      
      parkInfo[parkID]['Acres']=float(parkDfEntry[1][9])
      parkInfo[parkID]['Name']=parkDfEntry[1][6]
      parkInfo[parkID]['Borough']=parkDfEntry[1][2]
      parkInfo[parkID]['District']=parkDfEntry[1][4]
      parkInfo[parkID]['Category']=parkDfEntry[1][10]
      IF USING ABOVE, INDENT BELOW!
'''

    if parkInfo.get(parkID) == None:
      print 'Error: parkID %s not found in SITES file' % parkID
    elif Inspections.get(inspectionID) == None:
      print 'Error: inspectionID %s not found in FEATURERATING file' % inspectionID
    else:
      # Add inspection report to ParkID Dictionary
      parkInfo[parkID]['Inspections'][inspectionID] = Inspections[inspectionID]

      # Add inspection Date to Inspection sub Dictionary
      parkInfo[parkID]['Inspections'][inspectionID]['Date'] = date    # CHANGE TO DATETIME FORMAT!!!

      # !!!!!!   If Using commented out code from above, remove this section!
      # Create Ratio for inspection report and add to Inspection sub Dictionary
      inspectionFeatureCount = parkInfo[parkID]['Inspections'][inspectionID]['Count']
      inspectionFailureCount = parkInfo[parkID]['Inspections'][inspectionID]['Failures']
      parkInfo[parkID]['Inspections'][inspectionID]['Ratio'] = float(inspectionFailureCount)/inspectionFeatureCount



Error: inspectionID 77616 not found in FEATURERATING file
Error: inspectionID 77615 not found in FEATURERATING file
Error: inspectionID 75195 not found in FEATURERATING file
Error: inspectionID 74363 not found in FEATURERATING file
Error: inspectionID 72290 not found in FEATURERATING file
Error: inspectionID 71823 not found in FEATURERATING file
Error: inspectionID 71822 not found in FEATURERATING file
Error: inspectionID 71821 not found in FEATURERATING file
Error: inspectionID 71621 not found in FEATURERATING file
Error: inspectionID 71617 not found in FEATURERATING file
Error: inspectionID 71616 not found in FEATURERATING file
Error: inspectionID 71614 not found in FEATURERATING file
Error: inspectionID 71132 not found in FEATURERATING file
Error: inspectionID 65988 not found in FEATURERATING file
Error: parkID M276 not found in SITES file
Error: parkID M276 not found in SITES file
Error: parkID M276 not found in SITES file
Error: parkID M108B not found in SITES file
Error: inspecti

In [5]:
import pickle as pkl
fopen = open('parkInfo.pkl','wb')
pkl.dump(parkInfo,fopen)
fopen.close()

['Count', 'Failures', 'Ratio', 'Date']

In [1]:
import os
if not os.path.exists('../.dat/parkInfo.pkl'):
    print 'hello'

hello


In [4]:
import pickle as pkl
fopen = open('../.dat/parkInfo.pkl')

In [5]:
parkInfo = pkl.load(fopen)
fopen.close()

In [10]:
parkInfo['Q099-15']['Inspections'][28173]['Date'].year

2004

In [14]:
parkInfo['Q099-15']['Inspections']

{28173: {'Count': 13,
  'Date': Timestamp('2004-02-20 00:00:00'),
  'Failures': 0,
  'Ratio': 0.0},
 32475: {'Count': 11,
  'Date': Timestamp('2004-09-09 00:00:00'),
  'Failures': 3,
  'Ratio': 0.2727272727272727},
 35213: {'Count': 10,
  'Date': Timestamp('2005-02-22 00:00:00'),
  'Failures': 0,
  'Ratio': 0.0},
 36913: {'Count': 11,
  'Date': Timestamp('2005-05-23 00:00:00'),
  'Failures': 0,
  'Ratio': 0.0},
 39753: {'Count': 11,
  'Date': Timestamp('2005-10-03 00:00:00'),
  'Failures': 0,
  'Ratio': 0.0},
 41995: {'Count': 11,
  'Date': Timestamp('2006-01-24 00:00:00'),
  'Failures': 0,
  'Ratio': 0.0},
 44677: {'Count': 11,
  'Date': Timestamp('2006-06-20 00:00:00'),
  'Failures': 2,
  'Ratio': 0.18181818181818182},
 46942: {'Count': 11,
  'Date': Timestamp('2006-10-25 00:00:00'),
  'Failures': 4,
  'Ratio': 0.36363636363636365},
 52847: {'Count': 10,
  'Date': Timestamp('2007-08-23 00:00:00'),
  'Failures': 1,
  'Ratio': 0.1},
 54801: {'Count': 11,
  'Date': Timestamp('2008-01-03

In [26]:
AverageRatio = []
for parkName, parkData in parkInfo.items():
    if parkData['Category'] in ['Green Street', 'Large Park', 'Small Park']:
        count = 0
        ratioSum = 0
        for inspectionID, inspectionData in parkData['Inspections'].items():
            if inspectionData['Date'].year == 2014:
                count += 1
                ratioSum += inspectionData['Ratio']
        if not count == 0:
            AverageRatio.append((parkName, ratioSum/count))
print AverageRatio


[(u'Q102-ZN01', 0.11538461538461539), (u'Q102-ZN03', 0.0), (u'Q102-ZN04', 0.0), (u'X045-ZN03', 0.13333333333333333), (u'X045-ZN02', 0.18181818181818182), (u'X045-ZN01', 0.19230769230769232), (u'Q387A', 0.0), (u'Q387B', 0.0), (u'Q387D', 0.0), (u'X018', 0.1), (u'Q102-ZN02', 0.05555555555555555), (u'Q050-ZN09', 0.0), (u'R135-01', 0.06666666666666667), (u'X010-10', 0.0), (u'X150I', 0.08333333333333333), (u'X150H', 0.0), (u'X150L', 0.3), (u'X150G', 0.0), (u'X150E', 0.08333333333333333), (u'B378', 0.2), (u'B379', 0.15000000000000002), (u'B375', 0.08333333333333333), (u'B376', 0.1), (u'B377', 0.18181818181818182), (u'B372', 0.14285714285714285), (u'B110', 0.038461538461538464), (u'B111', 0.025641025641025644), (u'B114', 0.0909090909090909), (u'B115', 0.1111111111111111), (u'B116', 0.0), (u'B117', 0.0), (u'B248', 0.08333333333333333), (u'B249', 0.05555555555555555), (u'X010A', 0.1111111111111111), (u'B240', 0.0), (u'Q018', 0.0), (u'Q019', 0.0), (u'Q014', 0.1111111111111111), (u'Q017', 0.055555

In [25]:
list(zip(*AverageRatio)[1])

[0.0,
 0.0,
 0.11538461538461539,
 0.14285714285714285,
 0.0,
 0.0,
 0.13333333333333333,
 0.18181818181818182,
 0.19230769230769232,
 0.0,
 0.0,
 0.0,
 0.1,
 0.0,
 0.05555555555555555,
 0.125,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.06666666666666667,
 0.0,
 0.0,
 0.0,
 0.08333333333333333,
 0.0,
 0.3,
 0.0,
 0.08333333333333333,
 0.2,
 0.15000000000000002,
 0.08333333333333333,
 0.1,
 0.18181818181818182,
 0.14285714285714285,
 0.0,
 0.038461538461538464,
 0.025641025641025644,
 0.0909090909090909,
 0.1111111111111111,
 0.0,
 0.0,
 0.08333333333333333,
 0.05555555555555555,
 0.1111111111111111,
 0.0,
 0.0,
 0.0,
 0.1111111111111111,
 0.05555555555555555,
 0.08333333333333333,
 0.20833333333333331,
 0.23093573093573094,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.25,
 0.09090909090909091,
 0.1,
 0.05,
 0.041666666666666664,
 0.08333333333333333,
 0.045454545454545456,
 0.0,
 0.08333333333333333,
 0.045454545454545456,
 0.19230769230769232,
 0.0,
 0.0,
 0.0,
 0.055555555555